In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from tqdm import tqdm
import librosa
from numba import cuda
import librosa.display
import gc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
from sklearn.preprocessing import LabelEncoder
from scipy import signal
from scipy.io import wavfile as wav
from numpy.lib import stride_tricks
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM, Dense, Dropout
from keras.models import Sequential
from keras.utils import to_categorical
import pandas as pd

In [ ]:
from PIL import Image

In [ ]:
os.listdir('./spectrograms/')

In [ ]:
image = Image.open('./spectrograms/1c9a423f.jpg')

In [ ]:
im2arr = np.array(image) # im2arr.shape: height x width x channel
# arr2im = Image.fromarray(im2arr)
# arr2im.show()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [ ]:
configuration = tf.compat.v1.ConfigProto()
configuration.gpu_options.per_process_gpu_memory_fraction = 0.5
tf.config.experimental.set_memory_growth(gpus[0],True)
session = tf.compat.v1.Session(config=configuration)

In [ ]:
df = pd.read_csv('instruments.csv')
# df.set_index('fname', inplace=True)

# for f in df.index:
#     signal, rate = librosa.load('./cleaned/'+f)
#     df.at[f, 'length'] = signal.shape[0]/rate

# classes = list(np.unique(df.label))
# class_dist = df.groupby(['label'])['length'].mean()

# n_samples = 2 * int(df['length'].sum()/0.1)
# prob_dist = class_dist /class_dist.sum()
# choices = np.random.choice(class_dist.index, p=prob_dist)



# fig, ax = plt.subplots()
# ax.set_title('Class Distribution', y=1.08)
# ax.pie(class_dist, labels=class_dist.index, autopct='%1.1f%%',
#        shadow=False, startangle=90)
# ax.axis('equal')
# plt.show()

In [ ]:
classname =df[df.fname == "5388d14d.wav"].label.values

In [ ]:
classname[0]

In [ ]:
def read_audio(conf, pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=conf.sampling_rate)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf.samples: # long enough
        if trim_long_data:
            y = y[0:0+conf.samples]
    else: # pad blank
        padding = conf.samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
    return y

In [ ]:
def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

In [ ]:
def read_as_melspectrogram(conf, pathname, trim_long_data, debug_display=False):
    x = read_audio(conf, pathname, trim_long_data)
    mels = audio_to_melspectrogram(conf, x)
    return mels

In [ ]:
class conf:
    sampling_rate = 44100
    duration = 2
    hop_length = 347*duration # to make time steps 128
    fmin = 20
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = n_mels * 20
    samples = sampling_rate * duration

In [ ]:
def save_image_from_sound(img_path):
    filename = rename_file(img_path)
    x = read_as_melspectrogram(conf, img_path, trim_long_data=True, debug_display=True)
    
    plt.imshow(x, interpolation='nearest')
    #plt.savefig('./spectrograms/' + filename)
    
#     plt.close()
#     del x
#     gc.collect()

In [ ]:
save_image_from_sound('./spectrograms/1c9a423f.jpg')

In [ ]:
def rename_file(img_name):
    img_name = img_name.split("/")[2]
    img_name = img_name[:-4]
    img_name += ".jpg"
    return img_name

In [ ]:

for i, fn in enumerate(os.listdir('./cleaned/')):
    print(i)
    path = './cleaned/' + fn
    save_image_from_sound(path)

In [ ]:
images = os.listdir('./spectrograms/')

In [ ]:
temp = images[0].split('.')

In [ ]:
wavFile = temp[0] + '.wav'

In [ ]:
wavFile

In [ ]:
def build_feats():
    X = []
    Y = []
    for image in tqdm(os.listdir('./spectrograms/')):
        print(image)
        X.append(mpimg.imread('./spectrograms/' + image))
        temp = image.split('.')
        wavFile = temp[0] + '.wav'
        
        classname =df[df.fname == wavFile].label.values
        Y.append(classname[0])
    
    X = np.asarray(X)
    return X,Y

In [ ]:
X,Y = build_feats()

In [ ]:
Y = np.asarray(Y)

In [ ]:
Y = Y.flatten()

In [ ]:
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)

In [ ]:
labels = tf.keras.utils.to_categorical(Y,num_classes=10)

In [ ]:
def get_conv_model():
    model = Sequential()
    model.add(Conv2D(64,kernel_size=3, activation = 'relu', input_shape=(288,432,3)))
    model.add(Conv2D(32,kernel_size=3, activation = 'relu'))
#     model.add(Conv2D(64,(3,3), activation = 'relu', strides=(1,1),
#                     padding ='same'))
#     model.add(Conv2D(128,(3,3), activation = 'relu', strides=(1,1),
#                     padding ='same'))
    model.add(MaxPool2D((2,2)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(32,activation='relu'))
#     model.add(Dense(64, activation ='relu'))
    model.add(Dense(10, activation='softmax'))
    model.summary()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
    print(type(model))
    return model

In [ ]:
conv_model = get_conv_model()

In [ ]:
labels

In [ ]:
X[3]

In [ ]:
cnnScores = conv_model.fit(X,labels,epochs=100, batch_size=20, shuffle=True, validation_split=0.1)

In [ ]:
cnnScores

In [ ]:
import matplotlib.pyplot as pyplot

In [ ]:
fig = pyplot.figure()
fig.suptitle('CNN LOSS', fontsize=14)
# pyplot.plot(cnnScores.history['val_loss'],label="CNN Train", color="red")
pyplot.plot(cnnScores.history['loss'],label="CNN Train", color="green")

pyplot.xlabel('epoch')
pyplot.ylabel('loss(CE)')
pyplot.legend()
pyplot.show()

In [ ]:
cuda.select_device(0)
cuda.close()